# Trying to come up with an accuracy measure.

During the competition, the final loss will be given using the Pinball loss.

![alternative text](pinball_loss.png)

So we just need to compare the predicted quantiles to the real raw score of the actual day, since y is always the same.

### How do we work with Quantiles?
[This](https://timeseriesreasoning.com/contents/introduction-to-the-quantile-regression-model/) and [this](https://www.statsmodels.org/devel/examples/notebooks/generated/quantile_regression.html) were pretty helpful in understanding how this works. It seems as, rather than making a forecast, this Quantile Regression just gives you alternate trajectories of what your data may look like, based on looking at particular quantiles. For example, in the linear world, ordinary least squares will give you a forecast line based on your data's mean, whilte all the other forecast lines will be based on a particular quantile, like in this image:  

![alternative text](quant_reg_plot.png)

### What does that mean for our code?
[smf.quantreg](https://www.statsmodels.org/stable/generated/statsmodels.formula.api.quantreg.html#statsmodels.formula.api.quantreg) lets us estimate a [quantile regression model](https://www.statsmodels.org/dev/generated/statsmodels.regression.quantile_regression.QuantReg.html) using iterative reweighted least squares. Concretely, this means that, given our data, it lets us fit models based on particular quantiles of our data. We can also predict using this model.

### What does "predict" mean in this case?
`smf.quantreg(forumla, data)` just calls `statsmodels.regression.quantile_regression.QuantReg.from_formula`, and this formula handles what is used as the inputs and the target output. The forumla equation of the regression model is in [Patsy](https://patsy.readthedocs.io/en/latest/quickstart.html) syntax, and creates two DesignMatrix objects, the first representing the left-hand side of our formula (y), and the second representing the right-hand side (x1, x2, ..., xn). I believe these are the `endog` (endogenous/response variable) and `exog` (exogenous/explanatory variable(s)), respectively, based on the QuantReg language/documentation. So, when we [predict](https://www.statsmodels.org/dev/generated/statsmodels.regression.quantile_regression.QuantReg.predict.html), it uses the exog of the fitted model by default, meaning it will use the features (x1, ..., xn) of the equation to predict a y', based on the new data we give it. This means that each prediction is based on the particular quantile the model was fit to.

### How can we use quantiles in time series forecasts?
I am not too familiar with Quantile Regressors but it seems as we will always need one for our setup. I don't know of any other APIs or any machine learning methods that gives us these quantiles but I'm sure they exist. In any case, assuming this model is okay, there are a couple immediate ways that come to mind about how we can use it to do our forecasts:

1. We take the quantile-fitted models we get, and use those to predict the quantile for the next days. The input would be, however, all the data for the next day that were used as the exog parameters for the regressor, aka the x values in the formula. The issue here is we obviously do not have access to the SolarDownwardRadiation and WindSpeed for the next day, for example, so we would have to get a predictor for those variables that works well.

2. We In this case, the creation of the quantile models is theoretically separate from our predictor, since it is just used to create the data we will predict on. This means if we get a good-enough quantile model (aka if we notice that the model itself doesnt change that much with every new day of data), then we can just keep that QuantReg model fixed and use it to generate the quantiles that will be trained on. In the most basic case, we do not need current values of SolarDownwardRadiation and WindSpeed to make a prediction, since we simply use the quantiles, but if we want to make use of [exogenous data](https://www.sktime.net/en/latest/examples/01_forecasting.html#1.2.3-Forecasters-that-can-make-use-of-exogeneous-data), I think we will need a weather predictor here as well. 

Fortunately, we can assume having access to solar and wind forecast, because we are able to request this for certain day, via [the api](https://api.rebase.energy/challenges/redoc#tag/Data/operation/get_solar_and_wind_forecast_challenges_data_solar_and_wind_forecast_get).

### Open Questions
1. Should we be predicting the actual target y somehow, and using this to generate the quantiles, or should we try to forecast the quantiles directly using previous quantiles? Note: In the former case, y should be similar to the 0.5th quantile, which is the conditional median, rather the mean aka y itself. In the latter case, the quantiles are generated by the real y's that were observed in the past.
2. The Getting Started notebook has df entries with both `ref_datetime` and `valid_datetime`. ref_datetime is the time from when values were calculated and valid_datetime is the real datetime. But from the "Forecast scoring" section of the Getting Started it seems like it is a forecasted value when a row has a ref_datetime entry that is farther in the past than valid_datetime. But ref_datetime only has diff values on 6 hour intervals... and has 96 associated valid_datetimes (2 day forecast at 30 minute intervals, it seems). I also verified that neither all values in the ref_datetime nor valid_datetime columns are unique. Does that mean that the only non-forecasted values are at `modelling_table['ref_datetime'] == modelling_table['valid_datetime']`?? Why did they build the QuantReg model using the predicted values??

In [19]:
import os
os.chdir('/home/eokoyomon/code/HEFTcom24/src')

import statsmodels.formula.api as smf
from dotenv import load_dotenv
from omegaconf import OmegaConf
import datetime as dt
import pandas as pd
import numpy as np

from config import QuantRegConfig
from loaders import get_local_data, load_module

The following `pinball_score` function is adapted from the one in the Getting Started jupyter notebook. There they compute the pinball score over the entire modelling_table, which I do not think makes much sense for our case. They generated quantiles for every entry in the dataframe and then calculated the pinball score. But in our case, I think we want to just have the pinball score of the day we are predicting. However, if we keep everything in terms of Dataframes and Series, the code mostly remains the same.

In [20]:
def pinball(y,q,alpha):
    return (y-q)*alpha*(y>=q) + (q-y)*(1-alpha)*(y<q)

def pinball_score(predicted_quantiles, target):
    score = list()
    for qu in range(10,100,10):
        score.append(pinball(y=target["total_generation_MWh"],
            q=predicted_quantiles[f"q{qu}"],
            alpha=qu/100).mean())
    return sum(score)/len(score)

In [171]:
# Pick your target day as a pandas timestamp.
TARGET_DAY = pd.to_datetime(np.datetime64("2023-09-17 14:30:00"), utc=True)
# Pick the size of the sliding window you want to use for the training data. -1 if you want to use everything earlier than that day.
TRAINING_WINDOW = -1
# Which model config to use
CONFIG = QuantRegConfig()

In [172]:
def separate_target_data(df, target_day, training_window):
    # If this is uncommented, values are only on a 6 hour interval. But not sure 
    # df = df[df['ref_datetime'] == df['valid_datetime']]
    df = df.sort_values(by=['ref_datetime', 'valid_datetime'])
    target = df.loc[(df['valid_datetime'] >= target_day) & (df['valid_datetime'] < target_day + pd.Timedelta(1, unit="day"))]
    training = df[df['valid_datetime'] < target_day]
    if training_window != -1:
        training = training.tail(training_window)
    return training.reset_index(), target

In [176]:
# Runs predictors and returns the predicted quantiles and the pinball score.
def run_predictor():
    load_dotenv()
    root = os.getenv("root")
    data_dir = os.path.join(root, "data/")

    inputs, target_data = separate_target_data(get_local_data(data_dir), TARGET_DAY, TRAINING_WINDOW)

    config = CONFIG
    model = load_module("model", config, inputs)
    forecast_models = dict()
    predicted_quantiles = pd.DataFrame(index=target_data.index)
    predicted_quantiles["valid_datetime"] = target_data["valid_datetime"]

    for q in range(10, 100, 10):
        print(f"Starting Predictions for q{q}")

        forecast_models[f"q{q}"] = model.fit(quantile=q / 100)
        predicted_quantiles[f"q{q}"] = forecast_models[f"q{q}"].predict(target_data)
        predicted_quantiles.loc[predicted_quantiles[f"q{q}"] < 0, f"q{q}"] = 0

    return predicted_quantiles, pinball_score(predicted_quantiles, target_data)

The pinball loss score is a bit inaccurate, because both the training and the test data contain forecasts...

In [177]:
quantiles, score = run_predictor()
print()
print(score)
quantiles

Starting Predictions for q10
Starting Predictions for q20
Starting Predictions for q30
Starting Predictions for q40
Starting Predictions for q50
Starting Predictions for q60
Starting Predictions for q70
Starting Predictions for q80
Starting Predictions for q90
162.604748036862


,valid_datetime,q10,q20,q30,q40,q50,q60,q70,q80,q90
1036041,2023-09-17 14:30:00+00:00,38.599000,50.188802,62.571006,68.138713,74.020052,81.045562,87.199232,92.113990,96.853232
1036061,2023-09-17 15:00:00+00:00,28.291355,41.385175,53.933486,58.729248,63.783004,69.954173,75.677999,80.266440,84.983800
1036081,2023-09-17 15:30:00+00:00,19.676587,30.940787,42.061168,46.156017,50.175643,55.791092,61.118364,65.422524,69.632876
1036101,2023-09-17 16:00:00+00:00,11.582763,20.615142,29.847188,33.723472,37.093736,42.342651,47.422437,51.719983,55.631762
1036121,2023-09-17 16:30:00+00:00,7.647384,17.298029,26.201234,30.542270,34.163656,39.316506,44.445169,49.073785,53.390635
...,...,...,...,...,...,...,...,...,...,...
1036912,2023-09-18 12:00:00+00:00,98.191479,125.611869,159.659159,189.498220,212.721767,234.984340,260.491950,278.908350,291.225192
1036932,2023-09-18 12:30:00+00:00,77.691363,103.467529,135.862436,163.610165,185.798119,207.302334,232.225319,249.861483,262.089064
1036952,2023-09-18 13:00:00+00:00,59.616561,83.825697,114.575218,140.346058,161.327047,181.968290,206.290057,223.173728,235.217669
1036972,2023-09-18 13:30:00+00:00,58.929350,84.561939,113.893562,136.407017,155.166491,173.092314,194.110266,208.785184,219.653868
